# 2 Aggregating Data

Now that you know how to transform your data, you'll want to know more about how to aggregate your data to make it more interpretable. You'll learn a number of functions you can use to take many observations in your data and summarize them, including count, group_by, summarize, ungroup, and slice_min/slice_max.

# Counting by region

The counties dataset contains columns for region, state, population, and the number of citizens, which we selected and saved as the counties_selected table. In this exercise, you'll focus on the region column.

counties_selected <- counties %>%
  select(county, region, state, population, citizens)
  
# Instructions:

- Use count() to find the number of counties in each region, using a second argument to sort in descending order.



In [ ]:
# Use count to find the number of counties in each region
counties_selected %>%
  select(region, state, population, citizens)

counties_selected %>%
count(region, sort = TRUE)

# Counting citizens by state

You can weigh your count by particular variables rather than finding the number of counties. In this case, you'll find the number of citizens in each state.

counties_selected <- counties %>%
  select(county, region, state, population, citizens)

# Instructions:

- Count the number of counties in each state, weighted based on the citizens column, and sorted in descending order.

In [ ]:
# Find number of counties per state, weighted by citizens, sorted in descending order
counties_selected %>%
  select(region, state, population, citizens)

counties_selected %>%
count(state, wt = citizens, sort = TRUE)

# Mutating and counting

You can combine multiple verbs together to answer increasingly complicated questions of your data. For example: "What are the US states where the most people walk to work?"

You'll use the walk column, which offers a percentage of people in each county that walk to work, to add a new column and count based on it.

counties_selected <- counties %>%
  select(county, region, state, population, walk)

# Instructions:

- Use mutate() to calculate and add a column called population_walk, containing the total number of people who walk to work in a county.
- Use a (weighted and sorted) count() to find the total number of people who walk to work in each state.

In [ ]:
counties_selected %>%
  # Add population_walk containing the total number of people who walk to work 
  mutate(population_walk = population * walk / 100) %>%
  # Count weighted by the new column
  count(state, wt = population_walk, sort = TRUE)

# Summarizing

The summarize() verb is very useful for collapsing a large dataset into a single observation.

counties_selected <- counties %>%
  select(county, population, income, unemployment)

# Instructions:

- Summarize the counties dataset to find the following columns: min_population (with the smallest population), max_unemployment (with the maximum unemployment), and average_income (with the mean of the income variable).

In [ ]:
counties_selected %>%
  # Summarize to find minimum population, maximum unemployment, and average income
  summarise(min_population = min(population), max_unemployment = max(unemployment), average_income = mean(income))

# Summarizing by 

Another interesting column is land_area, which shows the land area in square miles. Here, you'll summarize both population and land area by state, with the purpose of finding the density (in people per square miles).

counties_selected <- counties %>%
  select(state, county, population, land_area)

# Instructions:

- Group the data by state, and summarize to create the columns total_area (the total land_area in square miles) and total_population (with total population).



In [ ]:
counties_selected %>%
  # Group by state
  group_by(state) %>%
  # Find the total area and population
  summarize(total_area = sum(land_area),
            total_population = sum(population))

- Add a density column containing the total population per square mile, using the columns you already created, then arrange in descending order.

In [ ]:
counties_selected %>%
  group_by(state) %>%
  summarize(total_area = sum(land_area),
            total_population = sum(population)) %>%
  # Add a density column
  mutate(density = total_population / total_area) %>%
  # Sort by density in descending order
  arrange(desc(density))

# Summarizing by state and region

You can group by multiple columns instead of grouping by one. Here, you'll practice aggregating by state and region, and notice how useful it is for performing multiple aggregations in a row.

counties_selected <- counties %>%
  select(region, state, county, population)

# Instructions:

- Summarize to find the total population, as a column called total_pop, in each combination of region and state, grouped in that order.

In [ ]:
counties_selected %>%
  # Group and summarize to find the total population
  group_by(region, state) %>%
  summarize(total_pop = sum(population))

- Notice the tibble is still grouped by region; use another summarize() step to calculate two new columns: the average state population in each region (average_pop) and the median state population in each region (median_pop).

In [ ]:
counties_selected %>%
  # Group and summarize to find the total population
  group_by(region, state) %>%
  summarize(total_pop = sum(population)) %>%
  # Calculate the average_pop and median_pop columns 
  summarize(average_pop = mean(total_pop),
            median_pop = median(total_pop))

# Selecting a county from each region

Previously, you used the walk column, which offers a percentage of people in each county that walk to work, to add a new column and count to find the total number of people who walk to work in each county.

Now, you're interested in finding the county within each region with the highest percentage of citizens who walk to work.

counties_selected <- counties %>%
  select(region, state, county, metro, population, walk)

# Instructions:

- Find the county in each region with the highest percentage of citizens who walk to work.

In [ ]:
counties_selected %>%
  # Group by region
  group_by(region) %>%
  # Find the county with the highest percentage of people who walk to work
  slice_max(walk, n = 1)

# Finding the lowest-income state in each region

You've been learning to combine multiple dplyr verbs together. Here, you'll combine group_by(), summarize(), and slice_min() to find the state in each region with the highest income.

When you group by multiple columns and then summarize, it's important to remember that the summarize "peels off" one of the groups, but leaves the rest on. For example, if you group_by(X, Y) then summarize, the result will still be grouped by X.

counties_selected <- counties %>%
  select(region, state, county, population, income)

# Instructions:

- Calculate the average income (as average_income) of counties within each region and state (notice the group_by() has already been done for you).
- Find the state with the lowest average_income in each region.

In [ ]:
counties_selected %>%
  group_by(region, state) %>%
  # Calculate average income
  summarize(average_income = mean(income)) %>%
  # Find the lowest income state in each region
  slice_min(average_income, n = 1)

# Using summarize, slice_max, and count together

In this chapter, you've learned to use six dplyr verbs related to aggregation: count(), group_by(), summarize(), ungroup(), slice_max(), and slice_min(). In this exercise, you'll combine them to answer a question:

In how many states do more people live in metro areas than non-metro areas?

Recall that the metro column has one of the two values "Metro" (for high-density city areas) or "Nonmetro" (for suburban and country areas).

counties_selected <- counties %>%
  select(state, metro, population)

# Instructions:

- For each combination of state and metro, find the total population as total_pop.




In [ ]:
counties_selected %>%
  # Find the total population for each combination of state and metro
  group_by(state, metro) %>%
  summarize(total_pop = sum(population))

- Extract the most populated row from each state, which will be either Metro or Nonmetro.

In [ ]:
counties_selected %>%
  # Find the total population for each combination of state and metro
  group_by(state, metro) %>%
  summarize(total_pop = sum(population)) %>%
  # Extract the most populated row for each state
  slice_max(total_pop, n = 1)

- Ungroup, then count how often Metro or Nonmetro appears to see how many states have more people living in those areas.

In [ ]:
counties_selected %>%
  # Find the total population for each combination of state and metro
  group_by(state, metro) %>%
  summarize(total_pop = sum(population)) %>%
  # Extract the most populated row for each state
  slice_max(total_pop, n = 1) %>%
  # Count the states with more people in Metro or Nonmetro areas
  ungroup() %>% 
  count(metro)